In [2]:
import pandas as pd
import numpy as np
import requests

## TODO:

1. Charts within Investment 
    - GFCF 
    - Gov Investment 
    - Rail Investment 
2. Dummy Categories

# Investment Dummy Charts

In [3]:
df = pd.read_csv('qna.csv')

In [4]:
s_df = pd.DataFrame(df.iloc[0]).reset_index()
s_df.columns = ['Series', 'Code']

s_df = s_df.iloc[1:]



In [5]:
s_df[s_df.Series.str.startswith("GFCF:Private sector")]

,Series,Code
657,"GFCF:Private sector: TOTAL: Dwellings, excludi...",L62T
658,GFCF:Private sector: TOTAL: Transfer of owners...,L62U
661,"GFCF:Private sector: TOTAL: Dwellings, excludi...",L636
662,GFCF:Private sector: TOTAL: Transfer of owners...,L637


# UKEA

In [3]:
df = pd.read_csv("ukea.csv")
s_df = pd.DataFrame(df.iloc[0]).reset_index()
s_df.columns = ['Series', 'Code']

s_df[s_df.Series.str.lower().str.contains("gross fixed")].to_excel("gfcf_codes.xlsx", index=False)

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_32778/4091454294.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [4]:
s_df

,Series,Code
0,Title,CDID
1,CG: Other miscellaneous current transfers D.75...,A29P
2,ESA:PC:LIFLOW:Currency and deposits,A4FK
3,Non-financial corporations sector (S11): Curre...,A4VR
4,Non-financial corporations sector (S11): Other...,A4VS
...,...,...
5684,10 Education CP SA £m,ZWUS
5685,10 Education CVM NAYear SA £m,ZWUT
5686,ESA:PC:LI:LEVEL:Direct Investment Loans,ZYBN
5687,ESA:PC:Liab:Level:Direct investment loans,ZYBO


In [8]:
df.columns

Index(['Title', 'CG: Other miscellaneous current transfers D.759u: CPSA',
       'ESA:PC:LIFLOW:Currency and deposits',
       'Non-financial corporations sector (S11): Currency and deposits (F.2): Flow: Liability: Current price: £m: Not seasonally adjusted',
       'Non-financial corporations sector (S11): Other deposits (F.29): Flow: Liability: Current price: £million: Not seasonally adjusted',
       'Households (S.14): Net lending (+) / Net borrowing (-) of the non-financial account (B.9n): Assets: Current price: £m: NSA',
       'BoP: current account balance as per cent of GDP',
       'Households (S.14): Net lending (+) / Net borrowing (-) of the non-financial account (B.9n): Assets: Current price: £m: SA',
       'NPISH (S.15): Net lending (+) / Net borrowing (-) of the non-financial account (B.9n): Assets: Current price: £m: NSA',
       'NPISH (S.15): Net lending (+) / Net borrowing (-) of the non-financial account (B.9n): Assets: Current price: £m: SA',
       ...
       'IMF

In [9]:
def parse_quarters(df):
    df['quarter'] = pd.to_numeric(df.quarter.str.replace('Q', ''))
    df['month'] = (df.quarter-1)*3+1
    df['date'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-01')
    return df

def parse_years(df):
    df['date'] = pd.to_datetime(df.year.astype(str) + '-01-01')
    return df

In [10]:
def clean_ons_series(dataset_id, series_id, date_parser, key):
    url = f'https://api.ons.gov.uk/timeseries/{timeseries_id}/dataset/{dataset_id}/data'
    req = requests.get(url)
    data = req.json()
    data = data[key]
    df = pd.DataFrame(data)
    df = date_parser(df)
    df = df[['date', 'value']]
    df = df.dropna()
    return df

# Investment

## GFCF

In [28]:
dataset_id = 'cxnv'
timeseries_id = 'npqt'

clean_ons_series(dataset_id, timeseries_id, parse_quarters, 'quarters').to_csv("cached_data/investment/gfcf.csv", index=False)

### Central Gov GFCF

In [15]:
timeseries_id = 'RNCZ'
dataset_id = 'UKEA'

gov_gfcf = clean_ons_series(dataset_id, timeseries_id, parse_quarters, 'quarters')
gov_gfcf.to_csv("cached_data/investment/central_gov_gfcf.csv", index=False)

In [16]:
gov_gfcf

,date,value
0,1987-01-01,2130
1,1987-04-01,2048
2,1987-07-01,2126
3,1987-10-01,2235
4,1988-01-01,2569
...,...,...
142,2022-07-01,13852
143,2022-10-01,15429
144,2023-01-01,16794
145,2023-04-01,15787


Private GFCF

In [49]:
timeseries_id = "NPEK"
dataset_id = "UKEA"

clean_ons_series(dataset_id, timeseries_id, parse_quarters, 'quarters').to_csv("cached_data/investment/private_gfcf.csv", index=False)

R&D

In [25]:
timeseries_id = "GLBH"
dataset_id = "GERD"

r_d_data = clean_ons_series(dataset_id, timeseries_id, parse_years, 'years')
r_d_data['value'] = r_d_data['value'].astype(float)
r_d_data = r_d_data.query("value > 0")
r_d_data.to_csv("cached_data/investment/r_d_gdp.csv", index=False)

In [24]:
r_d_data

,date,value
0,1981-01-01,2.01
2,1983-01-01,1.85
4,1985-01-01,1.84
5,1986-01-01,1.85
6,1987-01-01,1.76
7,1988-01-01,1.71
8,1989-01-01,1.72
9,1990-01-01,1.73
10,1991-01-01,1.67
11,1992-01-01,1.66


In [59]:
timeseries_id = "DLEI"
dataset_id = "BERD"

clean_ons_series(dataset_id, timeseries_id, parse_years, 'years').to_csv("cached_data/investment/business_rd.csv", index=False)

# 

In [57]:
## GFCF

timeseries_id = "DLEI"
dataset_id = "BERD"
key = 'years'
date_parser = parse_years


url = f'https://api.ons.gov.uk/timeseries/{timeseries_id}/dataset/{dataset_id}/data'
req = requests.get(url)
data = req.json()
data = data[key]
df = pd.DataFrame(data)

df = date_parser(df)
df = df[['date', 'value']]
df

df

,date,value
0,1981-01-01,195
1,1982-01-01,
2,1983-01-01,185
3,1984-01-01,
4,1985-01-01,171
5,1986-01-01,185
6,1987-01-01,181
7,1988-01-01,181
8,1989-01-01,171
9,1990-01-01,166


# Economic Inactivity

In [13]:
timeseries_id = "LF2S"
dataset_id = "lms"

clean_ons_series(dataset_id, timeseries_id, parse_quarters, 'quarters').to_csv("cached_data/human_capital/economic_inactivity_16_64.csv", index=False)

In [14]:
clean_ons_series(dataset_id, timeseries_id, parse_quarters, 'quarters')

,date,value
0,1971-01-01,24.9
1,1971-04-01,24.9
2,1971-07-01,25.1
3,1971-10-01,25.2
4,1972-01-01,24.8
...,...,...
207,2022-10-01,21.7
208,2023-01-01,21.5
209,2023-04-01,21.4
210,2023-07-01,21.9


In [ ]:

data = data[key]
df = pd.DataFrame(data)
df = date_parser(df)
df = df[['date', 'value']]
df

In [23]:
df

,date,label,month,quarter,sourceDataset,updateDate,value,year
0,1955-01-01,1955 Q1,1,1,CXNV,2022-08-11T23:00:00.000Z,14969,1955
1,1955-04-01,1955 Q2,4,2,CXNV,2022-08-11T23:00:00.000Z,15324,1955
2,1955-07-01,1955 Q3,7,3,CXNV,2022-08-11T23:00:00.000Z,15861,1955
3,1955-10-01,1955 Q4,10,4,CXNV,2022-08-11T23:00:00.000Z,16205,1955
4,1956-01-01,1956 Q1,1,1,CXNV,2022-12-22T00:00:00.000Z,16461,1956
...,...,...,...,...,...,...,...,...
271,2022-10-01,2022 Q4,10,4,CXNV,2023-12-22T00:00:00.000Z,106689,2022
272,2023-01-01,2023 Q1,1,1,CXNV,2024-02-15T00:00:00.000Z,109400,2023
273,2023-04-01,2023 Q2,4,2,CXNV,2024-02-15T00:00:00.000Z,108454,2023
274,2023-07-01,2023 Q3,7,3,CXNV,2024-02-15T00:00:00.000Z,106955,2023


In [14]:
data

{'description': {'cdid': 'NPQT',
  'contact': {'email': 'gcf@ons.gov.uk ',
   'name': 'Alison McCrae',
   'telephone': '+44 1633 455250'},
  'datasetId': 'CXNV',
  'datasetUri': '/economy/grossdomesticproductgdp/datasets/businessinvestment',
  'date': '2023 Q4',
  'nextRelease': '28 March 2024',
  'number': '108474',
  'preUnit': '',
  'releaseDate': '2024-02-15T00:00:00.000Z',
  'sampleSize': '0',
  'source': '',
  'title': 'Total Gross Fixed Capital Formation CVM SA £m',
  'unit': ''},
 'months': [],
 'quarters': [{'date': '1955 Q1',
   'label': '1955 Q1',
   'month': '',
   'quarter': 'Q1',
   'sourceDataset': 'CXNV',
   'updateDate': '2022-08-11T23:00:00.000Z',
   'value': '14969',
   'year': '1955'},
  {'date': '1955 Q2',
   'label': '1955 Q2',
   'month': '',
   'quarter': 'Q2',
   'sourceDataset': 'CXNV',
   'updateDate': '2022-08-11T23:00:00.000Z',
   'value': '15324',
   'year': '1955'},
  {'date': '1955 Q3',
   'label': '1955 Q3',
   'month': '',
   'quarter': 'Q3',
   'sourc

In [12]:
url

'http://api.beta.ons.gov.uk/v1/datasets/cxnv/timeseries/npqt'